In [9]:
import sys
import os
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))

import time
import talib
import sqlite3 as sqlite3
import yfinance as yf
import datetime
from util.util import get_ucodes

path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')

In [2]:
ucodes = get_ucodes()

for ucode in ucodes['美國ETF']:
    product = yf.Ticker(ucode)
    hist = product.history(period="max")
    hist = hist.drop(columns=['Dividends', 'Stock Splits'])
    hist.columns = ['open', 'high', 'low', 'close', 'volume']
    hist.insert(0, 'stime', hist.index.strftime('%Y-%m-%d'))
    hist.insert(1, 'code', ucode)
    hist = hist[['stime', 'code', 'close', 'high', 'low', 'open', 'volume']]

    conn = sqlite3.connect(path_sqlite)
    tb_name = 's_' + ucode.lower().replace('.', '')
    
    # conn.execute("DROP TABLE %s" % tb_name)
    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = "INSERT OR IGNORE INTO "+tb_name+" (stime, code, close, high, low, open, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, hist.values.tolist())
    conn.commit()
    conn.close()

In [15]:
# 指數
mapping = {
    '^IXIC': '.IXIC',
    '^DJI': '.DJI',
    '^GSPC': '.SP500',
    '^HSI': '.HSI',
    '^HSCE': '.HSCE',
    '000001.SS': '.SSEC',
    '399001.SZ': '.SZI',
    '^N225': '.N225', # 日
    '^TWII': '.TWII', # 台
    '^KS11': '.KS11', # 韓
    '^GDAXI': '.GDAXI', # 德DAX
    '^FCHI': '.FCHI', # 法CAC
    '^FTSE': '.FTSE' # 英富時100
}

for k, v in mapping.items():
    product = yf.Ticker(k)

    hist = product.history(period="max")
    hist = hist.drop(columns=['Dividends', 'Stock Splits'])
    hist.columns = ['open', 'high', 'low', 'close', 'volume']
    hist.insert(0, 'stime', hist.index.strftime('%Y-%m-%d'))
    hist.insert(1, 'code', v)
    hist = hist[['stime', 'code', 'close', 'high', 'low', 'open', 'volume']]
    print(product, hist.shape)
    
    conn = sqlite3.connect(path_sqlite)
    
    cur_time = datetime.datetime.now().strftime("%Y-%m-%d")
    conn.execute(f"INSERT OR IGNORE INTO name (code, nmll, stime) VALUES('{v}', '{product.info['shortName']}', '{cur_time}')")
    
    tb_name = 's_' + v.lower().replace('.', '')
    
    # conn.execute("DROP TABLE %s" % tb_name)
    conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
                 "high REAL, low REAL, open REAL, volume REAL)" % tb_name)
    
    stmt = "INSERT OR IGNORE INTO "+tb_name+" (stime, code, close, high, low, open, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
    conn.executemany(stmt, hist.values.tolist())
    conn.commit()
    
    time.sleep(2)

yfinance.Ticker object <^IXIC> (12379, 7)
yfinance.Ticker object <^DJI> (7464, 7)
yfinance.Ticker object <^GSPC> (19664, 7)
yfinance.Ticker object <^HSI> (8516, 7)
yfinance.Ticker object <^HSCE> (6879, 7)
yfinance.Ticker object <000001.SS> (5847, 7)
yfinance.Ticker object <399001.SZ> (5814, 7)
yfinance.Ticker object <^N225> (13705, 7)
yfinance.Ticker object <^TWII> (5922, 7)
yfinance.Ticker object <^KS11> (6084, 7)
yfinance.Ticker object <^GDAXI> (8476, 7)
yfinance.Ticker object <^FCHI> (7980, 7)
yfinance.Ticker object <^FTSE> (9559, 7)


In [4]:
product = yf.Ticker('2828.HK')
hist = product.history(period="max")
hist = hist.drop(columns=['Dividends', 'Stock Splits'])
hist.columns = ['open', 'high', 'low', 'close', 'volume']
hist.insert(0, 'stime', hist.index.strftime('%Y-%m-%d'))
hist.insert(1, 'code', '2828.HK')
hist = hist[['stime', 'code', 'close', 'high', 'low', 'open', 'volume']]

conn = sqlite3.connect(path_sqlite)
tb_name = 's_02828'

# conn.execute("DROP TABLE %s" % tb_name)
conn.execute("CREATE TABLE IF NOT EXISTS %s (stime TEXT PRIMARY KEY, code TEXT NOT NULL, close REAL, "
             "high REAL, low REAL, open REAL, volume REAL)" % tb_name)

stmt = "INSERT OR IGNORE INTO "+tb_name+" (stime, code, close, high, low, open, volume) VALUES (?, ?, ?, ?, ?, ?, ?)"
conn.executemany(stmt, hist.values.tolist())
conn.commit()
conn.close()